In [1]:
import os
import numpy as np
from dfply import *
import pandas as pd
from datetime import datetime

In [2]:
# 환결설정
pd.options.display.float_format = '{:,.0f}'.format

In [3]:
# 데이터 불러오기
금감원_금리시나리오 = pd.read_excel('./data/금감원_금리시나리오.xlsx') # 상승/하락/평행/경사 구분코드 넣기
일반_상품군정보 = pd.read_excel('./data/일반_상품군정보.xlsx', dtype={'PDGR_CD': str})
일반_약관정보 = pd.read_excel('./data/일반_약관정보.xlsx', dtype={'ARC_INPL_CD': str, 'PDC_CD': str, 'PDGR_CD': str})
일반_원수_개별추산액 = pd.read_excel('./data/일반_원수_개별추산액.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_원수_미경과보험료 = pd.read_excel('./data/일반_원수_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_납입예정보험료 = pd.read_excel('./data/일반_납입예정보험료.xlsx', dtype={'BSE_YM': str, 'ARC_INPL_CD': str, 'PYM_EPCT_YM': str})
일반_출재_개별추산액 = pd.read_excel('./data/일반_출재_개별추산액.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_출재_미경과보험료 = pd.read_excel('./data/일반_출재_미경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_원수_직전1년경과보험료 = pd.read_excel('./data/일반_원수_직전1년경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_출재_직전1년경과보험료 = pd.read_excel('./data/일반_출재_직전1년경과보험료.xlsx', dtype={'PDGR_CD': str, 'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'INER_CHN_DVCD': str, 'PDC_CD': str})
일반_연동수수료특약코드 = pd.read_excel('./data/일반_연동수수료특약코드.xlsx')
일반_진전추이 = pd.read_excel('./data/일반_진전추이.xlsx', dtype={'BSE_YM': str, 'PDGR_CD': str, 'AY_YM': str})
국가_중분류코드 = pd.read_excel('./data/국가_중분류코드.xlsx', dtype={'CNTR_CATG_CD': str})
국가_대분류코드 = pd.read_excel('./data/국가_대분류코드.xlsx', dtype={'CNTR_CATG_CD': str, 'KICS_CNTR_CATG_CD': str})

#### 1. 일반/원수/계약 - 미경과보험료

In [4]:
# 일반 원수 계약 PDGR_NM 가공
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 원수 계약 DMFR_DVCD 가공
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [28]:
# 일반 원수 계약 미경과보험료 집계
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(OGL_URND_PRM=np.sum(X.OGL_URND_PRM)) \
    >> mutate(OGL_URND_PRM=X.OGL_URND_PRM/1e3)
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료_집계.reset_index(drop=True)
일반_원수_미경과보험료_집계

,DMFR_DVCD,PDGR_NM,OGL_URND_PRM
0,국내,일반/근재,"6,997,300"
1,해외,일반/근재,"4,386,875"
2,국내,일반/기술,"29,494,169"
3,해외,일반/기술,"1,771,534"
4,국내,일반/기타,"38,447,007"
5,해외,일반/기타,"623,939"
6,국내,일반/농작물재해보상,"7,347,601"
7,국내,일반/배상,"66,489,198"
8,해외,일반/배상,"14,435,185"
9,국내,일반/보증,"11,066"


#### 2. 일반/출재/계약 - 미경과보험료

In [6]:
# 일반 출재 계약 PDGR_NM 가공
일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 계약 DMFR_DVCD 가공
일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [7]:
# 일반 출재 계약 미경과보험료 집계
일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(RN_URND_PRM=np.sum(X.RN_URND_PRM)) \
    >> mutate(RN_URND_PRM=X.RN_URND_PRM/1e3)
일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료_집계.reset_index(drop=True)
일반_출재_미경과보험료_집계

,DMFR_DVCD,PDGR_NM,RN_URND_PRM
0,국내,일반/근재,"1,544,819"
1,국내,일반/기술,"15,964,608"
2,국내,일반/기타,"26,986,882"
3,국내,일반/농작물재해보상,"4,408,561"
4,국내,일반/배상,"35,863,497"
5,국내,일반/보증,"10,513"
6,국내,일반/상해,"33,038,313"
7,국내,일반/외국인상해,0
8,국내,일반/종합,"43,807,620"
9,해외,일반/종합,"40,358,437"


#### 3. 할인율

In [8]:
금감원_금리시나리오_가공 = 금감원_금리시나리오 \
    >> mutate(MATURITY=X.BUCKET_END_DATE.dt.year*12 + X.BUCKET_END_DATE.dt.month - X.AS_OF_DATE.dt.year*12 - X.AS_OF_DATE.dt.month) \
    >> mutate(DISCOUNT=1/(1+X.VALUE)**(X.MATURITY/12)) \
    >> select(X.MATURITY, X.DISCOUNT)

#### 4. 일반/원수/계약 - 미래보험료

In [9]:
# 일반 계약 납입예정보험료 MATURITY 가공
일반_납입예정보험료_가공 = 일반_납입예정보험료 >> left_join(일반_약관정보[['ARC_INPL_CD', 'PDGR_CD']], on='ARC_INPL_CD') \
    >> mutate(PYM_EPCT_YM=pd.to_datetime(일반_납입예정보험료['PYM_EPCT_YM'], format='%Y%m')) \
    >> mutate(BSE_YM=pd.to_datetime(일반_납입예정보험료['BSE_YM'], format='%Y%m')) \
    >> mutate(MATURITY=X.PYM_EPCT_YM.dt.year*12 + X.PYM_EPCT_YM.dt.month - X.BSE_YM.dt.year*12 - X.BSE_YM.dt.month) \
    >> drop(X.BSE_YM, X.PYM_EPCT_YM) \
    >> mutate(MATURITY=if_else(X.MATURITY < 0, 0, X.MATURITY))

# 일반 계약 납입예정보험료 DMFR_DVCD 가공
일반_납입예정보험료_가공 = 일반_납입예정보험료_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

# 일반 계약 납입예정보험료 PDGR_NM 가공
일반_납입예정보험료_가공 = 일반_납입예정보험료_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDC_CD=X.ARC_INPL_CD.str.slice(0,5)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 할인율 반영
일반_납입예정보험료_가공 = 일반_납입예정보험료_가공 >> left_join(금감원_금리시나리오_가공, on='MATURITY') \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.WNCR_CNV_EPCT_PRM*X.DISCOUNT) \
    >> mutate(WNCR_CNV_RN_PRM_DISC=X.WNCR_CNV_RN_PRM*X.DISCOUNT)

In [10]:
# 일반 계약 납입예정보험료 집계
일반_납입예정보험료_집계 = 일반_납입예정보험료_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(WNCR_CNV_EPCT_PRM_DISC=np.sum(X.WNCR_CNV_EPCT_PRM_DISC), WNCR_CNV_RN_PRM_DISC=np.sum(X.WNCR_CNV_RN_PRM_DISC)) \
    >> mutate(WNCR_CNV_EPCT_PRM_DISC=X.WNCR_CNV_EPCT_PRM_DISC/1e3, WNCR_CNV_RN_PRM_DISC=X.WNCR_CNV_RN_PRM_DISC/1e3)
일반_납입예정보험료_집계 = 일반_납입예정보험료_집계.reset_index(drop=True)
일반_납입예정보험료_집계

,DMFR_DVCD,PDGR_NM,WNCR_CNV_EPCT_PRM_DISC,WNCR_CNV_RN_PRM_DISC
0,국내,일반/근재,"891,585","167,119"
1,국내,일반/기술,"1,290,419","642,444"
2,국내,일반/기타,"13,706","9,594"
3,국내,일반/배상,"2,877,078","843,306"
4,국내,일반/보증,"84,745","80,508"
5,국내,일반/상해,"14,336,685","2,883,026"
6,국내,일반/종합,"14,489,146","1,560,491"
7,국내,일반/해상,"25,565,519","10,440,624"


#### 5. 일반/원수/보상 - 개별추산액

In [11]:
# 일반 원수 보상 PDGR_NM 가공
일반_원수_개별추산액_가공 = 일반_원수_개별추산액 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 원수 보상 DMFR_DVCD 가공
일반_원수_개별추산액_가공 = 일반_원수_개별추산액_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [12]:
# 일반 원수 보상 개별추산액 집계
일반_원수_개별추산액_집계 = 일반_원수_개별추산액_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)
일반_원수_개별추산액_집계 = 일반_원수_개별추산액_집계.reset_index(drop=True)
일반_원수_개별추산액_집계

,DMFR_DVCD,PDGR_NM,OST_AMT
0,국내,일반/근재,"8,516,242"
1,해외,일반/근재,"4,176,001"
2,국내,일반/기술,"28,608,543"
3,해외,일반/기술,"925,406"
4,국내,일반/기타,"16,228,978"
5,국내,일반/농작물재해보상,"3,831,950"
6,해외,일반/농작물재해보상,0
7,국내,일반/배상,"46,768,724"
8,해외,일반/배상,"48,319,847"
9,국내,일반/보증,"251,736"


#### 6. 일반/출재/보상 - 개별추산액

In [13]:
# 일반 출재 보상 PDGR_NM 가공
일반_출재_개별추산액_가공 = 일반_출재_개별추산액 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 보상 DMFR_DVCD 가공
일반_출재_개별추산액_가공 = 일반_출재_개별추산액_가공 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(DMFR_DVCD=if_else(X.NTNL_CTRY_CD == 'KR', '국내', '해외'))

In [14]:
# 일반 출재 보상 개별추산액 집계
일반_출재_개별추산액_집계 = 일반_출재_개별추산액_가공 >> group_by(X.PDGR_NM, X.DMFR_DVCD) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT)) \
    >> mutate(RN_OST_AMT=X.RN_OST_AMT/1e3)
일반_출재_개별추산액_집계 = 일반_출재_개별추산액_집계.reset_index(drop=True)
일반_출재_개별추산액_집계

,DMFR_DVCD,PDGR_NM,RN_OST_AMT
0,국내,일반/근재,"1,771,726"
1,국내,일반/기술,"18,556,329"
2,국내,일반/기타,"6,564,878"
3,국내,일반/농작물재해보상,"2,394,430"
4,해외,일반/농작물재해보상,0
5,국내,일반/배상,"26,342,298"
6,국내,일반/보증,"239,149"
7,국내,일반/상해,"1,220,335"
8,국내,일반/종합,"57,247,450"
9,해외,일반/종합,"43,298,811"


#### 7. 할인요소

In [15]:
일반_진전추이_가공 = 일반_진전추이.copy()

할인요소 = []
for PDGR_CD in 일반_진전추이_가공['PDGR_CD'].unique():
    일반_진전추이_보종 = 일반_진전추이_가공.query('PDGR_CD == @PDGR_CD').sort_values(by='AY_YM')
    할인요소_보종 = []
    m = len(일반_진전추이_보종) - 1
    for i in range(m):
        n = m - i
        지급보험금 = 0
        for j in range(n):
            지급보험금 += 일반_진전추이_보종.iloc[m-j, 3+j+1+i] - 일반_진전추이_보종.iloc[m-j, 3+j+i]
        평균지급기간 = i + 0.5
        지급보험금 = max(지급보험금, 0)
        할인요소_보종.append([평균지급기간, 지급보험금])
    할인요소_보종 = pd.DataFrame(할인요소_보종, columns=['준비금_평균지급기간', '지급보험금'])
    할인요소_보종['지급보험금_비중'] = 할인요소_보종['지급보험금']/할인요소_보종['지급보험금'].sum()
#     할인요소_보종['누적비율'] = (할인요소_보종['지급보험금'] / 할인요소_보종['지급보험금'].sum()).cumsum()
    할인요소_보종['누적비율'] = 할인요소_보종['지급보험금_비중'].cumsum()
    할인요소_보종['MATURITY'] = (할인요소_보종['준비금_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소_보종.insert(0, 'PDGR_CD', PDGR_CD)
    할인요소_보종['보험료_평균지급기간'] = 할인요소_보종['준비금_평균지급기간'] - 0.2071
    할인요소_보종['조정비율'] = 0
    for i in range(m):
        if i==m-1:
            할인요소_보종['조정비율'].iloc[i] = 1
        else:
            할인요소_보종['조정비율'].iloc[i] = 할인요소_보종['누적비율'].iloc[i]+0.2071*(할인요소_보종['누적비율'].iloc[i+1]-할인요소_보종['누적비율'].iloc[i])
    할인요소_보종['1-조정비율'] = 0
    for i in range(m):
        if i==0:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]
        else:
            할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]-할인요소_보종['조정비율'].iloc[i-1]
    할인요소_보종['MATURITY'] = np.round(할인요소_보종['보험료_평균지급기간']*12).astype(int)
    할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공, on='MATURITY', how='left')
    할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
    할인요소.append(할인요소_보종)
할인요소 = pd.concat(할인요소).reset_index(drop=True)
할인요소 = 할인요소.merge(일반_상품군정보, on='PDGR_CD', how='left').drop('PDGR_CD', axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
# 할인요소 집계
보험료부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['지급보험금_비중']*df['DISCOUNT_x']).sum())
준비금부채_할인요소 = 할인요소.groupby('PDGR_NM').apply(lambda df: (df['1-조정비율']*df['DISCOUNT_y']).sum())
할인요소_집계 = pd.concat([보험료부채_할인요소, 준비금부채_할인요소], axis=1)
할인요소_집계.columns = ['할인요소_보험료', '할인요소_준비금']
할인요소_집계 = 할인요소_집계.reset_index()
할인요소_집계

,PDGR_NM,할인요소_보험료,할인요소_준비금
0,일반/근재,1,1
1,일반/기술,1,1
2,일반/기타,1,1
3,일반/배상,1,1
4,일반/상해,1,1
5,일반/종합,1,1
6,일반/해상,1,1
7,일반/화재,1,1


#### 8. 일반/출재/계약 - 직전 1년 경과보험료

In [17]:
# 일반 출재 계약 KICS_CNTR_CATG_NM 가공
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 출재 계약 PDGR_NM 가공
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 계약 PROP_YN_DVCD 가공
## 재보험특약코드 테이블을 가져오는 것과 차이가 있을 수 있음. 수정 요망.
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료_가공 >> mutate(RN_KND_CD=if_else(X.RRNR_TTY_CD.isin(['U2016KRQ1043001000', 'U2017KRQ1043001000', 'U2017KRS1040001000', 'U2018KRQ1043001000']), 'TP', X.RN_KND_CD)) \
    >> mutate(PROP_YN_DVCD=if_else(X.RN_KND_CD.isin(['TN', 'FN']), '출재/비비례', '출재/비례'))
    
# 일반 출재 계약 CMSN_DVCD 가공
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료_가공 >> mutate(CMSN_DVCD=if_else(X.RRNR_TTY_CD.isin(일반_연동수수료특약코드['RRNR_TTY_CD']), '연동', '비연동'))

In [18]:
# 일반 출재 계약 직전1년경과보험료 집계
일반_출재_직전1년경과보험료_집계 = 일반_출재_직전1년경과보험료_가공 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM).astype(float)) \
    >> mutate(RN_ELP_PRM=X.RN_ELP_PRM/1e3)
일반_출재_직전1년경과보험료_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,RN_ELP_PRM
0,국내,비연동,출재/비례,일반/근재,"382,874"
1,국내,연동,출재/비례,일반/근재,"3,331,849"
2,국내,비연동,출재/비비례,일반/근재,"13,845"
3,국내,비연동,출재/비례,일반/기술,"14,036,993"
4,국내,연동,출재/비례,일반/기술,"2,035,803"
5,국내,비연동,출재/비비례,일반/기술,"1,058,500"
6,국내,비연동,출재/비례,일반/기타,"61,358,948"
7,국내,비연동,출재/비례,일반/농작물재해보상,"11,846,188"
8,국내,비연동,출재/비비례,일반/농작물재해보상,"811,278"
9,국내,비연동,출재/비례,일반/배상,"52,656,719"


#### 9. 일반/원수/계약 - 직전 1년 경과보험료

In [19]:
# 일반 출재 계약 연동수수료비중
일반_출재_직전1년경과보험료_연동수수료비중 = 일반_출재_직전1년경과보험료_집계 >> group_by(X.KICS_CNTR_CATG_NM, X.CMSN_DVCD, X.PDGR_NM) \
    >> summarise(RN_ELP_PRM=np.sum(X.RN_ELP_PRM))
일반_출재_직전1년경과보험료_연동수수료비중 = 일반_출재_직전1년경과보험료_연동수수료비중 >> group_by(X.PDGR_NM, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_ELP_PRM_PDGR=np.sum(X.RN_ELP_PRM)) \
    >> right_join(일반_출재_직전1년경과보험료_연동수수료비중, on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(WEIGHT=if_else(X.RN_ELP_PRM==0, 0, X.RN_ELP_PRM/X.RN_ELP_PRM_PDGR))

In [20]:
# 일반 원수 계약 KICS_CNTR_CATG_NM 가공
일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 원수 계약 PDGR_NM 가공
일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# PROP_YN_DVCD 가공
일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료_가공 >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD.isin(['02', '03']), '수재/비례', np.nan)) \
    >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD=='01', '원수', X.PROP_YN_DVCD)) \
    >> mutate(PROP_YN_DVCD=if_else(X.INRI_RN_KND_CD.isin(['TN', 'FN']), '수재/비비례', X.PROP_YN_DVCD))

# 일반 원수 직전1년경과보험료 배분 (1)
일반_원수_직전1년경과보험료_배분1 = 일반_원수_직전1년경과보험료_가공 >> mask((X.RN_DVCD=='X') | (X.PROP_YN_DVCD!='원수')) \
    >> mutate(CMSN_DVCD='비연동')

# 일반 원수 직전1년경과보험료 배분 (2)
일반_원수_직전1년경과보험료_배분2 = 일반_원수_직전1년경과보험료_가공 >> mask(X.RN_DVCD=='O', X.PROP_YN_DVCD=='원수') \
    >> left_join(일반_출재_직전1년경과보험료_연동수수료비중[['KICS_CNTR_CATG_NM', 'PDGR_NM', 'CMSN_DVCD', 'WEIGHT']], on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(CMSN_DVCD=if_else(X.CMSN_DVCD.isna(), '비연동', X.CMSN_DVCD)) \
    >> mutate(WEIGHT=if_else(X.WEIGHT.isna(), 1, X.WEIGHT)) \
    >> mutate(ELP_PRM=X.ELP_PRM*X.WEIGHT)

In [21]:
# 일반 원수 직전1년경과보험료 배분
일반_원수_직전1년경과보험료_집계1 = 일반_원수_직전1년경과보험료_배분1 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM).astype(float)) \
    >> mutate(ELP_PRM=X.ELP_PRM/1e3)

일반_원수_직전1년경과보험료_집계2 = 일반_원수_직전1년경과보험료_배분2 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM)) \
    >> mutate(ELP_PRM=X.ELP_PRM/1e3)

일반_원수_직전1년경과보험료_집계 = pd.concat([일반_원수_직전1년경과보험료_집계1, 일반_원수_직전1년경과보험료_집계2]) \
    >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(ELP_PRM=np.sum(X.ELP_PRM))
일반_원수_직전1년경과보험료_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,ELP_PRM
0,국내,비연동,수재/비례,일반/근재,"709,806"
1,신흥국,비연동,수재/비례,일반/근재,"519,345"
2,국내,비연동,원수,일반/근재,"1,815,583"
3,미국&캐나다,비연동,원수,일반/근재,"6,952,884"
4,국내,연동,원수,일반/근재,"15,248,194"
...,...,...,...,...,...
57,국내,비연동,원수,일반/해상,"71,067,193"
58,미국&캐나다,비연동,원수,일반/해상,"312,225"
59,국내,비연동,원수,일반/화재,"9,329,176"
60,미국&캐나다,비연동,원수,일반/화재,"22,061,142"


#### 10. 일반/출재/보상 - 개별추산액 (2)

In [22]:
# 일반 출재 보상 KICS_CNTR_CATG_NM 가공
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 출재 보상 PDGR_NM 가공
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액2_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10900', '10901']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# 일반 출재 보상 PROP_YN_DVCD 가공
## 재보험특약코드 테이블을 가져오는 것과 차이가 있을 수 있음. 수정 요망.
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액2_가공 >> mutate(PROP_YN_DVCD='출재')
    
# 일반 출재 보상 CMSN_DVCD 가공
일반_출재_개별추산액2_가공 = 일반_출재_개별추산액2_가공 >> mutate(CMSN_DVCD=if_else(X.RRNR_TTY_CD.isin(일반_연동수수료특약코드['RRNR_TTY_CD']), '연동', '비연동'))

In [23]:
# 일반 출재 보상 개별추산액 집계
일반_출재_개별추산액2_집계 = 일반_출재_개별추산액2_가공 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT).astype(float)) \
    >> mutate(RN_OST_AMT=X.RN_OST_AMT/1e3)
일반_출재_개별추산액2_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,RN_OST_AMT
0,국내,비연동,출재,일반/근재,"600,637"
1,국내,연동,출재,일반/근재,"1,171,089"
2,국내,비연동,출재,일반/기술,"17,731,345"
3,국내,연동,출재,일반/기술,"824,984"
4,국내,비연동,출재,일반/기타,"6,564,878"
5,국내,비연동,출재,일반/농작물재해보상,"2,394,430"
6,신흥국,비연동,출재,일반/농작물재해보상,0
7,국내,비연동,출재,일반/배상,"22,242,933"
8,국내,연동,출재,일반/배상,"4,099,365"
9,국내,비연동,출재,일반/보증,"239,149"


#### 11. 일반/원수/보상 - 개별추산액 (2)

In [24]:
# 일반 출재 보상 연동수수료비중
일반_출재_개별추산액2_연동수수료비중 = 일반_출재_개별추산액2_집계 >> group_by(X.KICS_CNTR_CATG_NM, X.CMSN_DVCD, X.PDGR_NM) \
    >> summarise(RN_OST_AMT=np.sum(X.RN_OST_AMT))
일반_출재_개별추산액2_연동수수료비중 = 일반_출재_개별추산액2_연동수수료비중 >> group_by(X.PDGR_NM, X.KICS_CNTR_CATG_NM) \
    >> summarise(RN_OST_AMT_PDGR=np.sum(X.RN_OST_AMT)) \
    >> right_join(일반_출재_개별추산액2_연동수수료비중, on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(WEIGHT=if_else(X.RN_OST_AMT_PDGR==0, 0, X.RN_OST_AMT/X.RN_OST_AMT_PDGR))

In [25]:
# 일반 원수 보상 KICS_CNTR_CATG_NM 가공
일반_원수_개별추산액2_가공 = 일반_원수_개별추산액 >> mutate(NTNL_CTRY_CD=if_else(X.NTNL_CTRY_CD.isna(), 'KR', X.NTNL_CTRY_CD)) \
    >> mutate(CNTR_CD=X.NTNL_CTRY_CD) \
    >> left_join(국가_중분류코드, by='CNTR_CD') \
    >> left_join(국가_대분류코드, by='CNTR_CATG_CD') \
    >> drop(['CNTR_CD', 'CNTR_CATG_CD', 'KICS_CNTR_CATG_CD'])

# 일반 원수 계약 PDGR_NM 가공
일반_원수_개별추산액2_가공 = 일반_원수_개별추산액2_가공 >> left_join(일반_상품군정보, by='PDGR_CD') \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10902', '10903']), '일반/농작물재해보상', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10607']), '일반/외국인상해', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDC_CD.isin(['10011', '10013', '10016', '10017']), '일반/종합', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['일반/권원']), '일반/기타', X.PDGR_NM)) \
    >> mutate(PDGR_NM=if_else(X.PDGR_NM.isin(['보증']), '일반/보증', X.PDGR_NM))

# PROP_YN_DVCD 가공
일반_원수_개별추산액2_가공 = 일반_원수_개별추산액2_가공 >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD.isin(['02', '03']), '수재', np.nan)) \
    >> mutate(PROP_YN_DVCD=if_else(X.RRNR_DAT_DVCD=='01', '원수', X.PROP_YN_DVCD))

# 일반 원수 직전1년경과보험료 배분 (1)
일반_원수_개별추산액2_배분1 = 일반_원수_개별추산액2_가공 >> mask((X.RN_DVCD=='X') | (X.PROP_YN_DVCD!='원수')) \
    >> mutate(CMSN_DVCD='비연동')

# 일반 원수 직전1년경과보험료 배분 (2)
일반_원수_개별추산액2_배분2 = 일반_원수_개별추산액2_가공 >> mask(X.RN_DVCD=='O', X.PROP_YN_DVCD=='원수') \
    >> left_join(일반_출재_직전1년경과보험료_연동수수료비중[['KICS_CNTR_CATG_NM', 'PDGR_NM', 'CMSN_DVCD', 'WEIGHT']], on=['KICS_CNTR_CATG_NM', 'PDGR_NM']) \
    >> mutate(CMSN_DVCD=if_else(X.CMSN_DVCD.isna(), '비연동', X.CMSN_DVCD)) \
    >> mutate(WEIGHT=if_else(X.WEIGHT.isna(), 1, X.WEIGHT)) \
    >> mutate(OST_AMT=X.OST_AMT*X.WEIGHT)

In [26]:
# 일반 원수 직전1년경과보험료 배분
일반_원수_개별추산액2_집계1 = 일반_원수_개별추산액2_배분1 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT).astype(float)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)

일반_원수_개별추산액2_집계2 = 일반_원수_개별추산액2_배분2 >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT)) \
    >> mutate(OST_AMT=X.OST_AMT/1e3)

일반_원수_개별추산액2_집계 = pd.concat([일반_원수_개별추산액2_집계1, 일반_원수_개별추산액2_집계2]) \
    >> group_by(X.PDGR_NM, X.PROP_YN_DVCD, X.CMSN_DVCD, X.KICS_CNTR_CATG_NM) \
    >> summarise(OST_AMT=np.sum(X.OST_AMT))
일반_원수_개별추산액2_집계

,KICS_CNTR_CATG_NM,CMSN_DVCD,PROP_YN_DVCD,PDGR_NM,OST_AMT
0,국내,비연동,수재,일반/근재,"2,086,159"
1,국내,비연동,원수,일반/근재,"687,734"
2,미국&캐나다,비연동,원수,일반/근재,"4,176,001"
3,국내,연동,원수,일반/근재,"5,742,348"
4,국내,비연동,수재,일반/기술,0
5,기타선진국,비연동,수재,일반/기술,82
6,신흥국,비연동,수재,일반/기술,"326,130"
7,중국,비연동,수재,일반/기술,"4,141"
8,국내,비연동,원수,일반/기술,"25,209,235"
9,미국&캐나다,비연동,원수,일반/기술,"595,053"


In [27]:
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
now = datetime.now().strftime('%Y%m%d%H%M%S')
ver = 'v1.1.3'
with pd.ExcelWriter('./result/일반_현행추정부채_{}_{}.xlsx'.format(ver, now)) as writer:
    일반_원수_미경과보험료_집계.to_excel(writer, '일반_원수_미경과보험료', index=False)
    일반_출재_미경과보험료_집계.to_excel(writer, '일반_출재_미경과보험료', index=False)
    일반_납입예정보험료_집계.to_excel(writer, '일반_납입예정보험료', index=False)
    일반_원수_개별추산액_집계.to_excel(writer, '일반_원수_개별추산액', index=False)
    일반_출재_개별추산액_집계.to_excel(writer, '일반_출재_개별추산액', index=False)
    할인요소_집계.to_excel(writer, '할인요소', index=False)
    일반_출재_직전1년경과보험료_집계.to_excel(writer, '일반_출재_직전1년경과보험료', index=False)
    일반_원수_직전1년경과보험료_집계.to_excel(writer, '일반_원수_직전1년경과보험료', index=False)
    일반_출재_개별추산액2_집계.to_excel(writer, '일반_출재_개별추산액2', index=False)
    일반_원수_개별추산액2_집계.to_excel(writer, '일반_원수_개별추산액2', index=False)